In [42]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.io import push_notebook
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [43]:
import pydss

## CIGRE LV European System

In [44]:
sys1 = pydss.pydss()  # Load data
sys1.read('bus4_1p_load.json')
sys1.pf()                            # execute power flow 

trafos 0.000133514404296875
lines 0.0005457401275634766
trafos2 0.0015106201171875
lines 2 0.001069784164428711
diag_2dsparse 0.0016558170318603516
sparse.lil_matrix 0.00028443336486816406
Y_lines_primitive -> Y_primitive_sp 0.004228115081787109
Y_trafos_primitive -> Y_primitive_sp 0.0021812915802001953
Ys calc 0.009951591491699219
inv_Y_ii 0.0010366439819335938


/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [45]:
%timeit pydss.pf_eval(sys1.params_pf)

1000 loops, best of 3: 424 µs per loop


In [38]:
sys1.get_v()      # post process voltages
sys1.get_i()      # post process currents

I_lines 0.003321409225463867
writing line dic 0.0006597042083740234


### Graph with obtained results

In [39]:
sys1.bokeh_tools()

p = figure(width=600, height=600,
           title='European LV Network (CIGRE)')#,   x_range=[-200,200], y_range=[-350, 50])

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size=15, color="navy", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

In [34]:
(206.059*np.exp(1j*np.deg2rad(32.103))-200.889*np.exp(1j*np.deg2rad(32.457)))/((1.15+1j*0.08)*8.2583/1000)

(540.76006349662964+141.19576434715168j)

In [35]:
sys1.buses

[{'U_kV': 20.0,
  'bus': 'SourceBus',
  'deg_an': 0.0,
  'deg_bn': -119.99999999999999,
  'deg_cn': 120.00000000000004,
  'deg_ng': 0.0,
  'p_a': 65581.521113212861,
  'p_b': 45700.041806034344,
  'p_c': 1.0571181244062998e-09,
  'pos_x': 359.9,
  'pos_y': 162.81599999999162,
  'q_a': -14906.354008802489,
  'q_b': 49342.08629846829,
  'q_c': 3.9452185498862355e-09,
  'units': 'm',
  'v_ab': 11000.254678869938,
  'v_an': 6351.0,
  'v_bc': 11000.254678869938,
  'v_bn': 6350.9999999999991,
  'v_ca': 11000.254678869942,
  'v_cn': 6351.0,
  'v_ng': 0.0},
 {'U_kV': 0.4,
  'bus': '1',
  'deg_an': 29.761335510866505,
  'deg_bn': -89.999999999999986,
  'deg_cn': 150.00000000000003,
  'deg_ng': 0.0,
  'p_a': 0.0,
  'p_b': 0.0,
  'p_c': 0.0,
  'pos_x': 359.66300000000047,
  'pos_y': 162.81599999999162,
  'q_a': 0.0,
  'q_b': -0.0,
  'q_c': 0.0,
  'units': 'm',
  'v_ab': 415.23338256569315,
  'v_an': 239.86518728147158,
  'v_bc': 416.00963149180859,
  'v_bn': 240.18327272727265,
  'v_ca': 416.2331

In [41]:
np.abs(sys1.V_node)

array([[ 6351.        ],
       [ 6351.        ],
       [ 6351.        ],
       [  217.03947789],
       [  239.86518728],
       [  240.18327273],
       [  240.18327273],
       [  236.05374063],
       [  240.58741078],
       [  241.42902132],
       [  242.77918768],
       [  247.74130582]])

In [ ]:
sys1.nodes

### Interaction with powers

In [7]:
s_0_3pn = sys1.pq_3pn # 
s_0_3p  = sys1.pq_3p  # 
s_0_1p  = sys1.pq_1p  #

p = figure(width=600, height=300,
           title='Voltage vs load powers', 
           #x_range = [50,-300], y_range = [180,800],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-300, 50],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-300, 50],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))


def update(load_factor=1.0):

    sys1.pq_3pn = load_factor*s_0_3pn
    sys1.pq_3p = load_factor*s_0_3p
    sys1.pq_1p = load_factor*s_0_1p
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

#p_grid = gridplot([[p], [p_2]])
show(p, notebook_handle=True)


In [8]:
from ipywidgets import interact
interact(update, load_factor=(-1,1.2, 0.01))

I_lines 0.001592397689819336
writing line dic 0.00022220611572265625


### Interaction with compensator currents

In [ ]:
p = figure(width=600, height=400,
           title='Voltage with compensation', 
           x_range = [50,-300], y_range = [180,250],
           x_axis_label='Distance (m)',
           y_axis_label='Voltage (V)')
source = ColumnDataSource(sys1.bus_data)

i_a_0 = np.abs(sys1.I_node[28,0])
i_b_0 = np.abs(sys1.I_node[29,0])
i_c_0 = np.abs(sys1.I_node[30,0])
s_0 = sys1.pq_3pn # 

sys1.pf()

cr = p.circle(source=source, x='y', y='v_an', size=15, color="red", alpha=0.5)
p.circle(source=source, x='y', y='v_bn', size=15, color="green", alpha=0.5)
p.circle(source=source, x='y', y='v_cn', size=15, color="blue", alpha=0.5)
p.line([-300, 50],[231*1.05,231*1.05], color='red', line_width=5)
p.line([-300, 50],[231*0.90,231*0.90], color='blue', line_width=5)
#p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))

def update(i_a=0.0,i_b=0.0,i_c=0.0):

    sys1.I_node[28,0] = (i_a + i_a_0)*np.exp(1j*np.angle(sys1.V_node[28,0] -np.pi/2 ))
    sys1.I_node[29,0] = (i_b + i_b_0)*np.exp(1j*np.angle(sys1.V_node[29,0] -np.pi/2 ))
    sys1.I_node[30,0] = (i_c + i_c_0)*np.exp(1j*np.angle(sys1.V_node[30,0] -np.pi/2 ))    
    sys1.pf()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()

    source.data = sys1.bus_data
    push_notebook()

show(p, notebook_handle=True)

In [ ]:
interact(update, 
         i_a=(-100,100, 1),
         i_b=(-100,100, 1),
         i_c=(-100,100, 1))

In [ ]:
sys1.I_node[0:100]*400
